In [ ]:
import os

# Define qual implementação de MapReduce usar: 'java' ou 'python'
MAPREDUCE_LANG = 'java'  # Altere para 'python' para usar a implementação em Python
os.environ['MAPREDUCE_LANG'] = MAPREDUCE_LANG

print(f"Usando implementação de MapReduce: {os.environ['MAPREDUCE_LANG'].upper()}")

# Lab 4: Recomendação de Vacinas

Este notebook executa o pipeline de recomendação de vacinas para cães e gatos. As células abaixo irão compilar o código MapReduce, ingerir dados do PostgreSQL com Sqoop, executar o job no Hadoop, verifica os resultados e armazena os dados resultantes.

## Compilando a Aplicação MapReduce

A primeira etapa é compilar nosso código-fonte Java em um arquivo `.jar` executável. A célula abaixo usa o Maven para isso. Ela navega até o diretório do projeto e executa o `mvn package`. A flag `-q` é para uma saída mais limpa.

In [ ]:
%%bash
if [ "$MAPREDUCE_LANG" == "java" ]; then
    cd vaccine-recommendation
    mvn package -q
    echo "JAR compilado com sucesso em: target/vaccine-recommendation-1.0-SNAPSHOT.jar"
fi

## Consultando caderneta e o ciclo vacinal

Execute a célula abaixo para se conectar novamente e fazer uma consulta `SELECT` com os dados inseridos anteriormente.

In [ ]:
import psycopg2
import os

# As credenciais e o host são baseados no arquivo docker-compose.txt
DB_HOST = "localhost" # Nome do serviço no Docker Compose
DB_NAME = "postgres"
DB_USER = "postgres"
DB_USER_PWD = "postgres"

try:
    conn = psycopg2.connect(host=DB_HOST, dbname=DB_NAME, user=DB_USER, password=DB_USER_PWD)
    cur = conn.cursor()
    
    cur.execute("""SELECT
        p.pet_id,
        p.species,
        p.birth_date,
        vr.vaccine_reference_id,
        vr.vaccine_name,
        vr.description,
        vr.target_species,
        vr.first_dose_age_months,
        vr.booster_interval_months,
        vr.mandatory,
        vc.application_date
    FROM pet p
    CROSS JOIN vaccine_reference vr
    LEFT JOIN vaccination_record vc ON p.pet_id = vc.pet_id AND vr.vaccine_reference_id = vc.vaccine_reference_id""")
    rows = cur.fetchall()
    
    print("Recomendações vacinais encontradas:")
    for row in rows:
        print(row)
        
except Exception as e:
    print(f"Ocorreu um erro: {e}")
finally:
    if 'conn' in locals() and conn is not None:
        cur.close()
        conn.close()

## Montar recomendação vacinal com base na Caderneta de vacinação

## 1. Ingestão da caderneta e o ciclo vacinal com Sqoop

Importar os dados da cadernata de vacinação e do ciclo vacinal do PostgreSQL para o HDFS. O comando `hdfs dfs -rm` é usado para remover o diretório de destino antes da importação, garantindo que possamos executar esta célula várias vezes sem erros.

In [ ]:
%%bash
INPUT_DIR=/petshop/input_vaccine_recommendation

# Remove o diretório de entrada se ele já existir
hdfs dfs -test -d $INPUT_DIR
if [ $? -eq 0 ]; then
    echo "Removendo diretório HDFS existente: $INPUT_DIR"
    hdfs dfs -rm -r $INPUT_DIR
fi

echo "Iniciando importação com Sqoop..."
sqoop import \
    --connect jdbc:postgresql://localhost:5432/postgres \
    --username postgres \
    --password postgres \
    --query "SELECT \
        p.pet_id, \
        p.species, \
        p.birth_date, \
        vr.vaccine_reference_id, \
        vr.vaccine_name, \
        vr.description, \
        vr.target_species, \
        vr.first_dose_age_months, \
        vr.booster_interval_months, \
        vr.mandatory, \
        vc.application_date \
    FROM pet p \
    CROSS JOIN vaccine_reference vr \
    LEFT JOIN vaccination_record vc ON p.pet_id = vc.pet_id AND vr.vaccine_reference_id = vc.vaccine_reference_id \
    WHERE \$CONDITIONS" \
    --target-dir $INPUT_DIR \
    --m 1 \
    --split-by p.pet_id

echo "\nImportação concluída. Verificando os dados no HDFS:"
hdfs dfs -ls $INPUT_DIR
hdfs dfs -cat $INPUT_DIR/part-m-00000 | head -n 5

## 2. Execução do Job MapReduce

Com os dados no HDFS, podemos executar nosso job MapReduce. A célula abaixo submete o código Java ou Python para o Hadoop. O resultado será salvo no diretório `/petshop/output_vaccine_recommendation`.

In [ ]:
%%bash
INPUT_DIR=/petshop/input_vaccine_recommendation
OUTPUT_DIR=/petshop/output_vaccine_recommendation

# Remove o diretório de saída se ele já existir
hdfs dfs -test -d $OUTPUT_DIR
if [ $? -eq 0 ]; then
    echo "Removendo diretório HDFS de saída existente: $OUTPUT_DIR"
    hdfs dfs -rm -r $OUTPUT_DIR
fi

if [ "$MAPREDUCE_LANG" == "java" ]; then
    JAR_PATH=vaccine-recommendation/target/vaccine-recommendation-1.0-SNAPSHOT.jar

    echo "Executando o job MapReduce (Java)..."
    hadoop jar $JAR_PATH -D log4j.logger.com.petshop.hadoop.RecommendationMapper=DEBUG $INPUT_DIR $OUTPUT_DIR
    
elif [ "$MAPREDUCE_LANG" == "python" ]; then
    MAPPER_PATH=vaccine-recommendation-python/mapper.py
    REDUCER_PATH=vaccine-recommendation-python/reducer.py

    echo "Executando o job MapReduce (Python)..."
    hadoop jar $HADOOP_HOME/share/hadoop/tools/lib/hadoop-streaming-*.jar \
        -file $MAPPER_PATH -mapper $MAPPER_PATH \
        -file $REDUCER_PATH -reducer $REDUCER_PATH \
        -input $INPUT_DIR \
        -output $OUTPUT_DIR
fi

echo "\nJob concluído!"

## 3. Verificação do Resultado

Vamos verificar o resultado processado no HDFS. O arquivo `part-r-00000` deve conter o `pet_id`, `vaccine_name`, `description`, `mandatory` e `suggested_date`.

In [ ]:
%%bash
OUTPUT_DIR=/petshop/output_vaccine_recommendation

echo "Conteúdo do diretório de saída:"
hdfs dfs -ls $OUTPUT_DIR

echo "\nResultado do processamento:"
hdfs dfs -cat $OUTPUT_DIR/part-r-00000

## 4. Gravando e consultando resultados no Redis

A etapa final na arquitetura real seria carregar este resultado no Redis para consulta rápida. O serviço `loader-py` faria isso automaticamente. Abaixo, simulamos como os dados seriam armazenados e consultados usando `redis-cli`.

In [ ]:
%%bash
OUTPUT_FILE=/petshop/output_vaccine_recommendation/part-r-00000
REDIS_HOST=localhost

# Verifica se o arquivo de resultado existe
hdfs dfs -test -e $OUTPUT_FILE
if [ $? -ne 0 ]; then
    echo "Erro: Arquivo de resultado não encontrado em $OUTPUT_FILE"
    exit 1
fi

echo "Limpando recomendações antigas no Redis..."
redis-cli -h $REDIS_HOST KEYS "recommendation:vaccine:pet:*" | xargs -r redis-cli -h $REDIS_HOST DEL

echo "Carregando novas recomendações do HDFS para o Redis..."
# Lê o arquivo do HDFS e processa linha por linha
hdfs dfs -cat $OUTPUT_FILE | while IFS=$'\t' read -r pet_id values; do
    # Extrai os valores (nome da vacina, descrição, obrigatoriedade e data sugerida)
    vaccine_name=$(echo $values | cut -d',' -f1)
    description=$(echo $values | cut -d',' -f2)
    mandatory=$(echo $values | cut -d',' -f3)
    sug_date=$(echo $values | cut -d',' -f4)
    
    # Monta e executa o comando HSET para o Redis
    echo "Inserindo recomendação para o pet_id: $pet_id para '$vaccine_name' no dia $sug_date"
    redis-cli -h $REDIS_HOST HSET "recommendation:vaccine:pet:$pet_id:$vaccine_name" \
        suggested_date "$sug_date" \
        description "$description" \
        mandatory "$mandatory"
done

echo "\nCarga de dados concluída."

echo "Todos os registros inseridos:"
# redis-cli -h $REDIS_HOST KEYS "recommendation:vaccine:pet:*"

for key in $(redis-cli -h $REDIS_HOST KEYS "recommendation:vaccine:pet:*");
  do echo "\n Key : '$key'" 
     redis-cli -h $REDIS_HOST HGETALL $key;
done

## 5. Gravando resultados no PostgreSQL

Com o resultado processado e validado, a próxima etapa é armazená-lo no PostgreSQL para consumo por outras aplicações. A célula abaixo lê o resultado do HDFS e o insere na tabela `vaccine_recommendation`.

In [ ]:
import psycopg2
import os
from subprocess import Popen, PIPE

# Database connection details
DB_HOST = "localhost"
DB_NAME = "postgres"
DB_USER = "postgres"
DB_USER_PWD = "postgres"

# HDFS output file
output_file = "/petshop/output_vaccine_recommendation/part-r-00000"

try:
    # Connect to the database
    conn = psycopg2.connect(host=DB_HOST, dbname=DB_NAME, user=DB_USER, password=DB_USER_PWD)
    cur = conn.cursor()

    # Truncate the table before inserting new data
    cur.execute("TRUNCATE TABLE vaccine_recommendation;")
    print("Table vaccine_recommendation truncated.")

    # Read the HDFS file
    process = Popen(["hdfs", "dfs", "-cat", output_file], stdout=PIPE)
    (stdout, stderr) = process.communicate()
    exit_code = process.wait()

    if exit_code == 0:
        # Decode the output and split into lines
        lines = stdout.decode("utf-8").strip().split('\n')
        
        print(f"Inserting {len(lines)} records into vaccine_recommendation...")
        
        # Process each line and insert into the database
        for line in lines:
            if not line:
                continue
                
            pet_id, values = line.split('\t')
            parts = values.split(',')
            vaccine_name = parts[0]
            suggested_date = parts[-1]
            mandatory = parts[-2].lower() == 'true'
            description = ','.join(parts[1:-2])
            
            # Prepare and execute the INSERT statement
            cur.execute(
                "INSERT INTO vaccine_recommendation (pet_id, vaccine_name, description, mandatory, suggested_date) VALUES (%s, %s, %s, %s, %s)",
                (int(pet_id), vaccine_name, description, mandatory, suggested_date)
            )
        
        # Commit the transaction
        conn.commit()
        print("Data successfully inserted into vaccine_recommendation.")

    else:
        print(f"Error reading HDFS file: {stderr}")

except Exception as e:
    print(f"An error occurred: {e}")
finally:
    if 'conn' in locals() and conn is not None:
        cur.close()
        conn.close()

## 6. Consultando dados inseridos no PostgreSQL

Execute a célula abaixo para se conectar novamente e fazer uma consulta `SELECT` para verificar se os dados foram inseridos corretamente na tabela `vaccine_recommendation`.

In [ ]:
try:
    conn = psycopg2.connect(host=DB_HOST, dbname=DB_NAME, user=DB_USER, password=DB_USER_PWD)
    cur = conn.cursor()
    
    cur.execute("SELECT * FROM vaccine_recommendation;")
    rows = cur.fetchall()
    
    print("Registros encontrados:")
    for row in rows:
        print(row)
        
except Exception as e:
    print(f"Ocorreu um erro: {e}")
finally:
    if 'conn' in locals() and conn is not None:
        cur.close()
        conn.close()